In [1]:
import pandas as pd
samples = pd.read_csv('/Users/Surya/Downloads/screen-tsv-download.tsv', header=None, delimiter='\t')
samples.columns = ['cell_type','tissue']
leuk_samples = samples[samples['tissue'].str.contains('bone', case=False, na=False)]
print("There are", len(leuk_samples), "samples taken from a bone element/bone marrow")


There are 34 samples taken from a bone element/bone marrow


### Look
Look at big wig for open chromatin, can load it into IGV

locate potenital binding sites upstream of the promoter.

Identify potential binding sites, where it COULD happen; looking for consensus cis-regulatory elements identified upstream

If it's not present in AML, then maybe which cancers can we see it happening.

# Right Now
Let's try and get a scraping of protein atlas, query all of those cell lines, to make sure we got 'em all. Try with hyphen, no space-all text, replace hyphen with space, make sure that exclude chronic and not case sensitive

Let's try and get a visual understanding of binding sites and how we can approach implementing bigWig download from API

##  Creating a Comprehensive Profile of Available Leukemia Cell Lines in ENCODE

Acute Lymphoblastic Leukemia Cell Lines: 

https://www.cell.com/trends/immunology/abstract/S1471-4906(20)30307-0 

https://www.nature.com/articles/s41467-022-29224-5

Acute Myleoid Leukemia Cell lines: 

https://www.proteinatlas.org/humanproteome/cell+line/leukemia 

https://pmc.ncbi.nlm.nih.gov/articles/PMC10049680/table/ijms-24-05377-t001/ 



In [2]:
samples_ALL = ['DND-41','Jurkat','Loucy']
samples_AML = ['DND-41','HL-60', 'NB4', 'CMK']


## Scraping Protein Atlas for Cell Lines to Query for all Leukemia Samples for Verification of Above
https://www.proteinatlas.org/humanproteome/cell+line/leukemia

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Open the webpage
driver.get(url)

# Wait for the page to load (adjust as needed for specific elements)
driver.implicitly_wait(10)

# Get the page source after rendering
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Locate and extract the desired script
script_tags = soup.find_all('script')
for script in script_tags:
    if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
        full_line = script.text
        print("Extracted Script:")
        print(full_line)
        break

# Close the browser
driver.quit()


Extracted Script:

        var plot = $('#celline_prio_leukemia_LAML').scatterPlot([{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"M

In [60]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the script tag containing the desired identifier
    script_tags = soup.find_all('script')
    for script in script_tags:
        # Look for the specific identifier in the script content
        if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
            # Extract the full line containing the scatterPlot data
            full_line = re.search(r"var plot = \$\('#celline_prio_leukemia_LAML'\)\.scatterPlot\((.*?)\);", script.text, re.DOTALL)
            # if full_line:
            full_line_text = full_line.group(1)  # Extract the full JSON-like data
            cell_lines = re.findall(r'"name":"(.*?)"', full_line_text)
            #Strip trailers
            cell_lines = [x.split(' ')[0].replace('\\','') for x in cell_lines]
            print('All Leukemia Cell Lines in Protein Atlas:',cell_lines)  


All Leukemia Cell Lines in Protein Atlas: ['Kasumi-6', 'MOLM-6', 'KO52', 'MUTZ-3', 'GDM-1', 'Kasumi-1', 'Mono-Mac-1', 'OCI-AML-5', 'KG-1', 'OCI-AML-2', 'SKM-1', 'SIG-M5', 'NOMO-1', 'P31/FUJ', 'AML-193', 'Mono-Mac-6', 'UKE-1', 'EoL-1', 'CMK', 'Loucy', 'U-937', 'ME-1', 'THP-1', 'MOLT-13', 'NALM-1', 'HNT-34', 'NB4', 'HL-60', 'MOLT-16', 'PLB-985', 'MHH-CALL-2', 'EM-2', 'MOLM-13', 'SET-2', 'MHH-CALL-4', 'MV4-11', 'SEM', 'PL-21', 'SKNO-1', 'HEL', 'LAMA-84', 'M-07e', 'MHH-CALL-3', 'KCL-22', 'TF-1', 'Kasumi-2', 'Ku812', 'KE-37', 'CML-T1', 'ML-2', 'MOLM-16', 'NCO2', 'SUP-B15', 'HEL', 'JK-1', 'P12-Ichikawa', 'NALM-19', 'PF-382', 'RPMI-8402', 'MOLT-3', 'OCI-AML-3', 'MUTZ-5', 'HMC-1', 'F-36P', 'TALL-1', 'CCRF-SB', 'RCH-ACV', 'Peer', 'JURKAT', 'JURL-MK1', 'Jurkat', 'KOPN-8', 'JVM-3', 'NALM-6', 'REH', 'ALL-SIL', 'MEC-1', 'DND-41', 'BDCM', 'BV-173', 'OCI-M1', 'MEG-01', 'MOLT-4', 'SUP-T11', 'HPB-ALL', 'SUP-T1', 'JM-1', '697', 'OCI-M2', 'RS4;11', 'K-562', 'KYO-1', 'HAP1']


In [61]:
cell_lines

['Kasumi-6',
 'MOLM-6',
 'KO52',
 'MUTZ-3',
 'GDM-1',
 'Kasumi-1',
 'Mono-Mac-1',
 'OCI-AML-5',
 'KG-1',
 'OCI-AML-2',
 'SKM-1',
 'SIG-M5',
 'NOMO-1',
 'P31/FUJ',
 'AML-193',
 'Mono-Mac-6',
 'UKE-1',
 'EoL-1',
 'CMK',
 'Loucy',
 'U-937',
 'ME-1',
 'THP-1',
 'MOLT-13',
 'NALM-1',
 'HNT-34',
 'NB4',
 'HL-60',
 'MOLT-16',
 'PLB-985',
 'MHH-CALL-2',
 'EM-2',
 'MOLM-13',
 'SET-2',
 'MHH-CALL-4',
 'MV4-11',
 'SEM',
 'PL-21',
 'SKNO-1',
 'HEL',
 'LAMA-84',
 'M-07e',
 'MHH-CALL-3',
 'KCL-22',
 'TF-1',
 'Kasumi-2',
 'Ku812',
 'KE-37',
 'CML-T1',
 'ML-2',
 'MOLM-16',
 'NCO2',
 'SUP-B15',
 'HEL',
 'JK-1',
 'P12-Ichikawa',
 'NALM-19',
 'PF-382',
 'RPMI-8402',
 'MOLT-3',
 'OCI-AML-3',
 'MUTZ-5',
 'HMC-1',
 'F-36P',
 'TALL-1',
 'CCRF-SB',
 'RCH-ACV',
 'Peer',
 'JURKAT',
 'JURL-MK1',
 'Jurkat',
 'KOPN-8',
 'JVM-3',
 'NALM-6',
 'REH',
 'ALL-SIL',
 'MEC-1',
 'DND-41',
 'BDCM',
 'BV-173',
 'OCI-M1',
 'MEG-01',
 'MOLT-4',
 'SUP-T11',
 'HPB-ALL',
 'SUP-T1',
 'JM-1',
 '697',
 'OCI-M2',
 'RS4;11',
 'K-562',

In [64]:
cell_lines = cell_lines[0:5]

In [71]:
cell_lines

['KO52', 'Kasumi-6', 'MOTM-13']

In [68]:
samples

,cell_type,tissue
0,22Rv1,prostate gland
1,22Rv1 treated with 10 nM 17?-hydroxy-5?-andros...,prostate gland
2,8988T,pancreas
3,A172,brain
4,A549,lung
...,...,...
1513,WERI-Rb-1,eye
1514,WI38,lung
1515,WI38 genetically modified using stable transfe...,connective tissue
1516,WI38 genetically modified using stable transfe...,connective tissue


In [70]:
import pandas as pd
import requests

def fetch_synonyms(cell_line):
    """
    Fetches the identifier and synonyms for a given cell line from the Cellosaurus API.

    Args:
        cell_line (str): The cell line to query.

    Returns:
        tuple: The identifier and a list of synonyms for the cell line.
    """
    url = f"https://api.cellosaurus.org/search/cell-line?q={cell_line}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data.get('cell-line-list', []):
            identifier = None
            synonyms = []
            for name_entry in entry.get('name-list', []):
                if name_entry['type'] == 'identifier':
                    identifier = name_entry['value']
                elif name_entry['type'] == 'synonym':
                    synonyms.append(name_entry['value'])
            return identifier, synonyms
    return None, []

def find_matching_rows(cell_lines, samples_df):
    """
    Matches rows in the samples DataFrame based on cell line names and synonyms.

    Args:
        cell_lines (list): List of cell lines to query.
        samples_df (pd.DataFrame): DataFrame containing "cell_type" and "tissue" columns.

    Returns:
        pd.DataFrame: DataFrame with matched rows and an additional column for the identifier.
    """
    matched_rows = []

    for cell_line in cell_lines:
        # Fetch identifier and synonyms for the cell line
        identifier, synonyms = fetch_synonyms(cell_line)
        if identifier:
            # Combine identifier and synonyms into one list of query strings
            query_strings = [identifier] + synonyms
            print(query_strings)
            
            # Match rows where cell_type contains any of the query strings
            for _, row in samples_df.iterrows():
                if any(query_string in row['cell_type'] for query_string in query_strings):
                    matched_row = row.to_dict()
                    matched_row['matched_identifier'] = identifier
                    matched_rows.append(matched_row)

    # Create a new DataFrame from the matched rows
    return pd.DataFrame(matched_rows)

# Example cell lines list
cell_lines = ["KO52", "Kasumi-6", "MOTM-13"]

# # Example samples DataFrame
# data = {
#     "cell_type": ["KO52-derived cells", "Kasumi-6 tumor cells", "Some unrelated cells"],
#     "tissue": ["Bone marrow", "Lymph node", "Liver"]
# }
# samples_df = pd.DataFrame(data)
samples_df = samples
# Find matching rows
matched_df = find_matching_rows(cell_lines, samples_df)

# Display the output DataFrame
print(matched_df)


Empty DataFrame
Columns: []
Index: []


In [29]:
name_entries = re.findall(r'"name":"(.*?)"', full_line_text)

In [41]:
samples

,cell_type,tissue
0,22Rv1,prostate gland
1,22Rv1 treated with 10 nM 17?-hydroxy-5?-andros...,prostate gland
2,8988T,pancreas
3,A172,brain
4,A549,lung
...,...,...
1513,WERI-Rb-1,eye
1514,WI38,lung
1515,WI38 genetically modified using stable transfe...,connective tissue
1516,WI38 genetically modified using stable transfe...,connective tissue


In [55]:
samples_df = samples
# Example array of cell lines
cell_lines = ["Kasumi-1", "MOLT-4", "K562", "Jurkat"]

# Function to fetch synonyms from Cellosaurus for a given cell line
def fetch_synonyms(cell_line):
    try:
        # Fetching data from Cellosaurus API
        response = requests.get(f"https://www.cellosaurus.org/CVCL_{cell_line}.txt")
        response.raise_for_status()
        # Extracting synonyms from the text
        synonyms = []
        for line in response.text.split('\n'):
            if line.startswith("Synonyms:"):
                synonyms += line.replace("Synonyms:", "").strip().split("; ")
        return synonyms
    except requests.exceptions.RequestException as e:
        print(f"Error fetching synonyms for {cell_line}: {e}")
        return []

# Initialize a dictionary to hold results
results = {}

# Loop through each cell line
for cell_line in cell_lines:
    # Fetch synonyms
    synonyms = fetch_synonyms(cell_line)
    # Add original cell line to the search terms
    search_terms = [cell_line] + synonyms
    search_terms = [term.split(' ')[0].replace('\\', '') for term in search_terms]  # Process as required

    # Search in the samples dataframe
    matches = samples_df[samples_df['cell_type'].str.contains('|'.join(search_terms), na=False, case=False)]

    # Store matches in the results dictionary
    results[cell_line] = matches

# Display results
for cell_line, matches in results.items():
    print(f"Matches for {cell_line}:")
    print(matches)

Error fetching synonyms for Kasumi-1: 404 Client Error: Not Found for url: https://www.cellosaurus.org/CVCL_Kasumi-1.txt
Error fetching synonyms for MOLT-4: 404 Client Error: Not Found for url: https://www.cellosaurus.org/CVCL_MOLT-4.txt
Matches for Kasumi-1:
Empty DataFrame
Columns: [cell_type, tissue]
Index: []
Matches for MOLT-4:
Empty DataFrame
Columns: [cell_type, tissue]
Index: []
Matches for K562:
                                             cell_type        tissue
749                                               K562  bodily fluid
750                                      K562 G1 phase         blood
751                                      K562 G2 phase         blood
752  K562 treated with 0.05% dimethyl sulfoxide for...         blood
753     K562 treated with 1 ?M vorinostat for 72 hours         blood
754  K562 treated with 500 ?M sodium butyrate for 7...         blood
Matches for Jurkat:
              cell_type        tissue
748  Jurkat; Clone E6-1  bodily fluid


In [54]:
import requests
import pandas as pd
import re

# List of cell line names extracted from your previous process
cell_lines = [
    "Kasumi-6", "MOLM-6", "KO52",  # Add other cell line names as needed
]

# Diseases of interest
diseases_of_interest = {
    "Acute myeloid leukemia", "AML", "Acute myelogenous leukemia",
    "Acute lymphoblastic leukemia", "Acute lymphocytic leukemia", "ALL"
}

# Initialize an empty DataFrame to store matching samples
matching_samples = pd.DataFrame()

# Iterate over each cell line name
for cell_line in cell_lines:
    # Query Cellosaurus API for the cell line
    response = requests.get(f"https://api.cellosaurus.org/search/cell-line?q={cell_line}")
    if response.status_code == 200:
        data = response.json()
        if data['number_of_results'] > 0:
            # Extract the first result
            cell_line_data = data['results'][0]
            # Retrieve synonyms
            synonyms = cell_line_data.get('sy', [])
            # Include the original cell line name
            search_terms = [cell_line] + synonyms
            # Retrieve associated diseases
            associated_diseases = cell_line_data.get('di', [])
            # Check if any associated disease matches the diseases of interest
            if any(disease in diseases_of_interest for disease in associated_diseases):
                # Search for each term in the 'cell_type' column of your samples DataFrame
                for term in search_terms:
                    # Create a regex pattern to search for the term as a whole word, case-insensitive
                    pattern = re.compile(rf'\b{re.escape(term)}\b', re.IGNORECASE)
                    # Find matching rows
                    matches = samples[samples['cell_type'].str.contains(pattern, na=False)]
                    # Append matches to the result DataFrame
                    matching_samples = pd.concat([matching_samples, matches])
    else:
        print(f"Failed to retrieve data for {cell_line}. Status code: {response.status_code}")

# Drop duplicate rows, if any
matching_samples = matching_samples.drop_duplicates()

# Output the matching samples
print(matching_samples)


KeyError: 'number_of_results'

In [ ]:
import requests
import re
import pandas as pd

# Sample DataFrame (replace this with your actual DataFrame)
data = {
    'cell_type': ['Kasumi-6', 'MOLM-6', 'OtherCellType'],
    'disease': ['Acute myeloid leukemia', 'AML', 'OtherDisease']
}
samples_df = pd.DataFrame(data)

# List of cell lines extracted from the JavaScript data
full_line_text = '''{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""}'''
cell_lines = re.findall(r'"name":"(.*?)"', full_line_text)
cell_lines = [x.split(' ')[0].replace('\\', '') for x in cell_lines]

# Diseases of interest
diseases_of_interest = {
    'acute myeloid leukemia',
    'aml',
    'acute myelogenous leukemia',
    'acute lymphoblastic leukemia',
    'acute lymphocytic leukemia',
    'all'
}

# Function to fetch synonyms and disease information from Cellosaurus
def fetch_cellosaurus_info(cell_line_name):
    url = f"https://api.cellosaurus.org/search/cell-line?q={cell_line_name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'cell_lines' in data and data['cell_lines']:
            cell_line_data = data['cell_lines'][0]
            synonyms = cell_line_data.get('synonyms', [])
            disease = cell_line_data.get('disease', '').lower()
            return synonyms, disease
    return [], ''

# List to store matching rows
matching_rows = []

# Iterate over each cell line
for cell_line in cell_lines:
    synonyms, disease = fetch_cellosaurus_info(cell_line)
    # Check if the disease is of interest
    if disease in diseases_of_interest:
        # Create a set of all possible names to search for
        search_terms = {cell_line.lower()} | {syn.lower() for syn in synonyms}
        # Search in the DataFrame
        for _, row in samples_df.iterrows():
            cell_type = row['cell_type'].lower()
            if any(term in cell_type for term in search_terms):
                matching_rows.append(row)

# Create a DataFrame with the matching rows
matching_df = pd.DataFrame(matching_rows)

# Display the matching DataFrame
print(matching_df)


Empty DataFrame
Columns: []
Index: []


In [26]:
import requests
from bs4 import BeautifulSoup
import re
import json

# URL of the Protein Atlas Leukemia page
url = "https://www.proteinatlas.org/humanproteome/cell+line/leukemia"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the script tag containing the desired identifier
    script_tags = soup.find_all('script')
    for script in script_tags:
        # Look for the specific identifier in the script content
        if "var plot = $('#celline_prio_leukemia_LAML')" in script.text:
            # Extract the full line containing the scatterPlot data
            full_line = re.search(r"var plot = \$\('#celline_prio_leukemia_LAML'\)\.scatterPlot\((.*?)\);", script.text, re.DOTALL)
            if full_line:
                full_line_text = full_line.group(1)  # Extract the JSON-like data
                
                # Extract all "name" entries using regex
                name_entries = re.findall(r'"name":"(.*?)"', full_line_text)
                
                # Remove duplicates and sort alphabetically
                unique_names = sorted(set(name_entries))
                
                # Output the list of names
                print("Extracted Cell Line Names:")
                print(unique_names)


Extracted Cell Line Names:
['697', 'ALL-SIL', 'AML-193', 'BDCM', 'BV-173', 'CCRF-SB', 'CMK', 'CML-T1', 'DND-41', 'EM-2', 'EoL-1', 'F-36P', 'GDM-1', 'HAP1', 'HEL', 'HEL 92.1.7', 'HL-60', 'HMC-1', 'HNT-34', 'HPB-ALL', 'JK-1', 'JM-1', 'JURKAT', 'JURL-MK1', 'JVM-3', 'Jurkat E6.1', 'K-562', 'KCL-22', 'KE-37', 'KG-1', 'KO52', 'KOPN-8', 'KYO-1', 'Kasumi-1', 'Kasumi-2', 'Kasumi-6', 'Ku812', 'LAMA-84', 'Loucy', 'M-07e', 'ME-1 [Human leukemia]', 'MEC-1', 'MEG-01', 'MHH-CALL-2', 'MHH-CALL-3', 'MHH-CALL-4', 'ML-2', 'MOLM-13', 'MOLM-16', 'MOLM-6', 'MOLT-13', 'MOLT-16', 'MOLT-3', 'MOLT-4', 'MUTZ-3', 'MUTZ-5', 'MV4-11', 'Mono-Mac-1', 'Mono-Mac-6', 'NALM-1', 'NALM-19', 'NALM-6', 'NB4', 'NCO2', 'NOMO-1', 'OCI-AML-2', 'OCI-AML-3', 'OCI-AML-5', 'OCI-M1', 'OCI-M2', 'P12-Ichikawa', 'P31\\/FUJ', 'PF-382', 'PL-21', 'PLB-985', 'Peer', 'RCH-ACV', 'REH', 'RPMI-8402', 'RS4;11', 'SEM', 'SET-2', 'SIG-M5', 'SKM-1', 'SKNO-1', 'SUP-B15', 'SUP-T1', 'SUP-T11', 'TALL-1 [Human adult T-ALL]', 'TF-1', 'THP-1', 'U-937', 'UK

In [25]:
if 'K-562' in unique_names:
    print(True)

True


In [ ]:
full_line_text

'[{"color":"lightgrey","data":[{"class":"cl938","name":"Kasumi-6","x":1.96068175909325,"y":0.834886071020256,"tooltip":"Kasumi-6<br>NES: 1.96<br>Corr: 0.83","url":""},{"class":"cl1095","name":"MOLM-6","x":1.90032457194471,"y":0.835674513933875,"tooltip":"MOLM-6<br>NES: 1.90<br>Corr: 0.84","url":""},{"class":"cl974","name":"KO52","x":1.98706550466945,"y":0.833892719414413,"tooltip":"KO52<br>NES: 1.99<br>Corr: 0.83","url":""},{"class":"cl1110","name":"MUTZ-3","x":1.88884309348927,"y":0.834852139753001,"tooltip":"MUTZ-3<br>NES: 1.89<br>Corr: 0.83","url":""},{"class":"cl694","name":"GDM-1","x":1.9503423496079,"y":0.823659272269025,"tooltip":"GDM-1<br>NES: 1.95<br>Corr: 0.82","url":""},{"class":"cl936","name":"Kasumi-1","x":1.86985160948779,"y":0.815727956855209,"tooltip":"Kasumi-1<br>NES: 1.87<br>Corr: 0.82","url":""},{"class":"cl1103","name":"Mono-Mac-1","x":1.8749012322619,"y":0.812513211118372,"tooltip":"Mono-Mac-1<br>NES: 1.87<br>Corr: 0.81","url":""},{"class":"cl1284","name":"OCI-AML-

---------------------------------------------------------------------------------
### Attempt at Scraping Screen Directly --> Do BIGWIG download from API directly 
---------------------------------------------------------------------------------

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')     

# Path to the chromedriver, make sure chromedriver is in your PATH or provide the full path
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', options=options)

try:
    # Navigate to SCREEN
    url = "https://screen.encodeproject.org/search/?q=EIF6&assembly=GRCh38&uuid=985f9f7f-730a-49d4-8dbf-ee36eab1ab43"
    driver.get(url)
    time.sleep(5)  # Wait for the page to fully load

    # Search for consensus cis-regulatory elements
    search_box = driver.find_element(By.XPATH, "//input[@type='search']")
    search_box.send_keys("EIF6")
    search_box.send_keys(Keys.RETURN)
    time.sleep(5)  # Wait for search results

    # Extract consensus cis-regulatory elements information
    results = driver.find_elements(By.CLASS_NAME, "result-item")
    for result in results:
        element_text = result.text
        if "TP53" in element_text:
            print("Found TP53 binding site information:\n")
            print(element_text)

finally:
    # Close the WebDriver
    driver.quit()

# Note: Adjust the XPATHs and CLASS_NAME according to the actual HTML structure of the SCREEN page.
# The sleep commands are used to wait for the page to load, but it's better to use explicit waits.


TypeError: __init__() got an unexpected keyword argument 'executable_path'